In [1]:
import os

os.environ['http_proxy'] = 'http://10.176.58.101:7890'
os.environ['https_proxy'] = 'http://10.176.58.101:7890'

import sys
sys.path.append('/remote-home1/jxwang/project/lorsa/src')

import torch
from transformers import GPTNeoXForCausalLM, GPTNeoXTokenizerFast
from transformer_lens import HookedTransformer
from transformer_lens.HookedTransformerConfig import HookedTransformerConfig
from transformer_lens.components import Attention
from datasets import load_from_disk
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR
torch.set_grad_enabled(False)

import copy

from tqdm import tqdm

import pysvelte
import numpy as np
import einops

from model.attention import LowRankSparseAttention
from config import LoRSATrainConfig, LoRSAConfig

In [3]:
import importlib
import config
import model.attention

importlib.reload(config)
importlib.reload(model.attention)

from model.attention import LowRankSparseAttention
from config import LoRSATrainConfig, LoRSAConfig

In [46]:
import importlib
importlib.reload(pysvelte)

<module 'pysvelte' from '/remote-home1/jxwang/project/lorsa/PySvelte/pysvelte/__init__.py'>

In [2]:
from config import LoRSATrainConfig, LoRSAConfig
cfg = LoRSAConfig(
    # meta head config
    d_qk_head = 16,
    d_ov_head = 1,
    n_qk_heads = 96,
    n_ov_heads = 1536,
    rotary_scale = 1,
    rotary_dim = 16,
    device = "cuda",
    virtual_kv_num=0,
    
    mode = 'top_k',
    top_k = 64,
    
    # orig head config
    model_name = 'EleutherAI/pythia-160m',
    layer = 5,
    max_length = 32,
    prepend_bos = True,
)

In [3]:
hf_model = GPTNeoXForCausalLM.from_pretrained(cfg.model_name)
tokenizer = GPTNeoXTokenizerFast.from_pretrained(cfg.model_name)
model = HookedTransformer.from_pretrained(cfg.model_name, hf_model=hf_model, tokenizer=tokenizer, device=cfg.device)
model.eval()
for param in model.parameters():
    param.requires_grad = False

# get original attention block
orig_attn = model.blocks[cfg.layer].attn

# load dataset
dataset_path = '/remote-home1/share/research/mechinterp/gpt2-dictionary/data/openwebtext'
dataset = load_from_disk(dataset_path)

Loaded pretrained model EleutherAI/pythia-160m into HookedTransformer


Loading dataset from disk:   0%|          | 0/80 [00:00<?, ?it/s]

In [4]:
batch_size = 32
data = DataLoader(dataset['text'], batch_size=batch_size)
data_iter = iter(data)

In [5]:
cfg.update_from_model_config(model.cfg)

# initialize lorsa
lorsa = LowRankSparseAttention(cfg).to(cfg.device)

lorsa_path = '/remote-home1/jxwang/project/lorsa/result/pythia-160m-sah/L5A-d16&1-n96&1536-k64.pth'

state_dict = torch.load(lorsa_path, map_location=cfg.device)

lorsa.load_state_dict(state_dict)

/tmp/ipykernel_4128146/2785011488.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(lorsa_path, map_location=cfg.device)


<All keys matched successfully>

In [6]:
from lm_saes import LanguageModelSAERunnerConfig, SAEConfig
from lm_saes.sae import SparseAutoEncoder

hook_point_in = 'A'
exp_factor = 32
layer = cfg.layer

ckpt_path = f"/remote-home1/share/research/mechinterp/pythia-160m-LX{hook_point_in}-{exp_factor}x-topk"
ckpt_path = os.path.join(ckpt_path, f"Pythia-160m-L{layer}{hook_point_in}-{exp_factor}x-lr0.001")
sae_config = SAEConfig.from_pretrained(ckpt_path)
sae_config.sae_pretrained_name_or_path = ckpt_path

sae = SparseAutoEncoder.from_config(cfg=sae_config)

In [32]:
ov_vector = lorsa.W_O.squeeze(1)
index = 0
feature_vector = ov_vector[index].unsqueeze(0)

sae_vector = sae.decoder.weight.permute(1, 0)

cos = nn.CosineSimilarity(dim=1)
res = cos(feature_vector, sae_vector)

top_cos, top_cos_indices  = torch.topk(res, 50)
print(top_cos)
print(top_cos_indices)

tensor([0.4211, 0.4056, 0.2865, 0.2809, 0.2313, 0.2043, 0.1942, 0.1914, 0.1907,
        0.1899, 0.1897, 0.1835, 0.1823, 0.1821, 0.1744, 0.1709, 0.1629, 0.1620,
        0.1598, 0.1590, 0.1585, 0.1577, 0.1568, 0.1560, 0.1550, 0.1537, 0.1534,
        0.1530, 0.1529, 0.1529, 0.1518, 0.1493, 0.1446, 0.1427, 0.1425, 0.1410,
        0.1409, 0.1396, 0.1392, 0.1382, 0.1380, 0.1378, 0.1378, 0.1364, 0.1363,
        0.1359, 0.1353, 0.1345, 0.1336, 0.1332], device='cuda:0')
tensor([ 4507, 19469, 12592, 22663, 19649, 20573, 21458,  5800, 19762,  1596,
        10693, 10243,  8632, 17760, 21158, 19916, 12826,  2710, 12912,   180,
        13872,  2279, 17383, 13326, 22224, 21433, 22492, 13408, 19695,   348,
        21343, 20197, 20819, 11060,  1808, 17498,  2566,  9014, 14252, 12498,
        11148, 14226,  2459,  7193, 23046,  4379,  1833,  6136,  5019, 20192],
       device='cuda:0')


In [31]:
import warnings

warnings.filterwarnings("ignore")

block_size = 32

for head_index in [0]: #tqdm(range(lorsa.cfg.n_ov_heads), desc = 'analysis'):
    batch_num = 2 ** 10
    top_num = 20

    hook_in_name = f'blocks.{cfg.layer}.ln1.hook_normalized'
    hook_out_name = f'blocks.{cfg.layer}.hook_attn_out'

    names_filter = [hook_in_name]
    
    top_z_info = []
    z_info = []
    tokens_info = []
    
    for _ in tqdm(range(batch_num), desc = 'batch'):
        # Run the forward pass
        batch = next(data_iter)
        tokens = model.to_tokens(batch, prepend_bos=cfg.prepend_bos).to(cfg.device)
        tokens = tokens[:, :cfg.max_length]
        
        _, cache = model.run_with_cache(tokens, names_filter=names_filter)
        hook_in = cache[hook_in_name]
        
        q, k, v = lorsa.cal_q_k_v(hook_in)
        pattern = lorsa.cal_pattern(q, k)
        
        v_ = einops.rearrange(
            v, "batch key_pos head_index d_head -> batch head_index key_pos d_head"
        )[:, head_index].squeeze(dim=-1) # Shape: (batch_size, key_pos)
        pattern_ = pattern[:, head_index // cfg.n_qk_heads] # Shape: (batch_size, query_pos, key_pos)
        z = pattern_[:, :, :] * v_[:, None, :] # Shape: (batch_size, query_pos, key_pos)
        
        '''
        out = lorsa.cal_out_with_h(hook_in)
        out_head = out[:, :, head_index, :]
        del out
        torch.cuda.empty_cache()
        out_head_l2 = torch.linalg.vector_norm(out_head, dim=-1)
        '''
        
        for idx in range(batch_size):
            for block in range(cfg.max_length // block_size):
                block_z = z[idx, block * block_size:(block + 1) * block_size, block * block_size:(block + 1) * block_size]
                block_top_z = torch.max(block_z)
                block_token = tokens[idx, block * block_size:(block + 1) * block_size]
                
                top_z_info.append(block_top_z)
                z_info.append(block_z)
                tokens_info.append(block_token)
        
        all_data = list(zip(top_z_info, z_info, tokens_info))
        all_data_sorted = sorted(all_data, key=lambda x: x[0], reverse=True)
        top_data = all_data_sorted[:top_num]
        top_z_info, z_info, tokens_info = zip(*top_data)
        
        # Convert back to lists for the next iteration
        top_z_info = list(top_z_info)
        z_info = list(z_info)
        tokens_info = list(tokens_info)
    
    top_z_info = torch.stack(top_z_info)
    tokens_info = torch.stack(tokens_info)
    z_info = torch.stack(z_info)
    
    html = pysvelte.Html(f"<h1>L{lorsa.cfg.layer}A OV_Head{head_index} Positive</h1>")
    for idx in range(top_num):
        strings = [model.to_string(token) for token in tokens_info[idx]]
        html += pysvelte.AttentionMulti(tokens=strings, attention=z_info[idx].unsqueeze(2) * 10)
        
        line_counter = 0
        line_html = ""

        for token_idx in range(len(strings)):
            z = z_info.sum(dim=-1)[idx][token_idx].item()
            s = f"{strings[token_idx]}"
            # Wrap each token in a span with fixed width
            # line_html += f'<span style="display:inline-block; width:150px; color: rgb(0, 0, 0); background-color: rgb(255, {255 - min(l2 * 100, 255)}, 0);">{s}</span>'
            line_html += f'<span style="display:inline-block; color: rgb(0, 0, 0); background-color: rgb(255, {255 - min(z * 200, 255)}, 255);">{s.replace(" ", "&nbsp;")}</span>'
            # line_counter += 1
            '''
            if line_counter == 16 or token_idx == len(strings) - 1:
                html += pysvelte.Html(f"<p>{line_html}</p>")
                line_html = ""
                line_counter = 0
            '''
        html += pysvelte.Html(f"<p>{line_html}</p>")
            
    html.publish(f'/remote-home1/jxwang/project/lorsa/result/html/L{lorsa.cfg.layer}A_OV_Head{head_index}_Positive.html')

batch: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [01:27<00:00, 11.67it/s]


In [30]:
print(top_z_info)
print(torch.max(z_info.sum(dim=-1), dim=-1))

tensor([1.9158, 1.9158, 1.9158, 1.9158, 1.9158, 1.9158, 1.9158, 1.9158, 1.9158,
        1.9158, 1.9158, 1.9158, 1.9158, 1.9158, 1.9158, 1.9158, 1.9158, 1.9158,
        1.9158, 1.9158], device='cuda:0')
torch.return_types.max(
values=tensor([0.0342, 0.1080, 0.4138, 0.7113, 0.3914, 2.4006, 0.3518, 0.3166, 0.2680,
        0.4974, 0.3387, 0.4381, 0.0566, 0.0069, 0.2876, 0.2058, 0.2360, 0.2201,
        0.2259, 0.7381], device='cuda:0'),
indices=tensor([31, 30,  7, 24, 14, 29, 23, 28, 31, 10, 24, 28,  3,  0, 15, 28, 11,  3,
        11, 20], device='cuda:0'))


In [17]:
print(z_info.shape)

torch.Size([20, 32, 32])


In [19]:
string = [model.to_string(token) for token in tokens_info[0]]
print(string)

['<|endoftext|>', 'Three', ' int', 're', 'pid', ' game', ' designers', ' just', ' sent', ' their', ' latest', ' board', ' game', ',', ' called', ' Secret', ' Hitler', ',', ' to', ' all', ' 100', ' U', '.', 'S', '.', ' senators', ',', ' because', ' “', 'although', ' the', ' game']


In [18]:
html = pysvelte.Html(f"<h1>L{lorsa.cfg.layer}A OV_Head{head_index} Positive</h1>")
for idx in range(top_num):
    strings = [model.to_string(token) for token in tokens_info[idx]]
    html += pysvelte.AttentionMulti(tokens=strings, attention=z_info[idx].unsqueeze(2) * 10)
    
    line_counter = 0
    line_html = ""

    for token_idx in range(len(strings)):
        z = z_info.sum(dim=-1)[idx][token_idx].item()
        s = f"{strings[token_idx]}"
        # Wrap each token in a span with fixed width
        # line_html += f'<span style="display:inline-block; width:150px; color: rgb(0, 0, 0); background-color: rgb(255, {255 - min(l2 * 100, 255)}, 0);">{s}</span>'
        line_html += f'<span style="display:inline-block; color: rgb(0, 0, 0); background-color: rgb(255, {255 - min(z * 100, 255)}, 255);">{s.replace(" ", "&nbsp;")}</span>'
        # line_counter += 1
        '''
        if line_counter == 16 or token_idx == len(strings) - 1:
            html += pysvelte.Html(f"<p>{line_html}</p>")
            line_html = ""
            line_counter = 0
        '''
    html += pysvelte.Html(f"<p>{line_html}</p>")
        
html.publish(f'/remote-home1/jxwang/project/lorsa/result/html/L{lorsa.cfg.layer}A_OV_Head{head_index}_Positive.html')

'/remote-home1/jxwang/project/lorsa/result/html/L5A_OV_Head6_Positive.html'

In [81]:
sentence = '''An Anglican priest charged last month with sexually assaulting five youths in Edmonton during the 1980s is facing more charges after police say more victims came forward.

In February, Edmonton police charged Gordon William Dominey, 63, with five counts of sexual assault and five counts of gross indecency in connection with incidents that allegedly occurred at the Edmonton Youth Development Centre from 1985 to 1989.

He now faces 18 sexual assault charges and nine gross indecency charges in relation to the alleged assaults, which are reported to have happened at the incarceration facility.

Dominey was employed at the centre at the time.

The new charges come after four more men came forward in the last month to report allegations of sexual assault at the facility during the same time period, Edmonton police say.

Dominey transferred from the Diocese of Edmonton to the Diocese of New Westminster in Vancouver in July of 1990.

He was the priest-in-charge at St. Catherine's Anglican Church in North Vancouver from the summer of 2015 until he was arrested in Coquitlam on Feb. 4, when the Bishop of New Westminster, Melissa Skelton, placed Dominey on administrative leave, according to a release.

Police, who'''

